In [ ]:
import pandas as pd

from utils import query_snowflake, load_into_snowflake

In [ ]:

df_am_budapest = query_snowflake("select * from ea_insights_assessment.ea_insights_schema.agent_master_budapest")
df_am_capetown = query_snowflake("select * from ea_insights_assessment.ea_insights_schema.agent_master_capetown")
df_am_manila = query_snowflake("select * from ea_insights_assessment.ea_insights_schema.agent_master_manila")
df_am_toronto = query_snowflake("select * from ea_insights_assessment.ea_insights_schema.agent_master_toronto")


In [5]:
def unnest_training_completed(df):
    values = [1, 2, 3]   # values you want to create columns for
    
    df_copy = df.copy()

    for v in values:
        df_copy[f"COMPLETED_L{v}"] = df_copy["TRAININGS_COMPLETED"].apply(lambda x: v in x)

    return df_copy

df_am_budapest["TENURE_MONTHS"] = df_am_budapest["TENURE"].str.extract(r"([-+]?\d*\.?\d+)").astype(float) * 12
df_am_budapest["TRAININGS_COMPLETED"] = df_am_budapest["TRAININGS_COMPLETED"].str.extractall(r"L(\d+)").astype(int).groupby(level=0)[0].apply(list)
df_am_budapest["CONTACT_CENTRE"] = "Budapest"
df_am_budapest_formatted = df_am_budapest[["AGENT_ID", "EMPLOYMENT_TYPE", "TRAININGS_COMPLETED", "TENURE_MONTHS", "CONTACT_CENTRE"]]
df_am_budapest_formatted = unnest_training_completed(df_am_budapest_formatted)

df_am_capetown["TENURE_MONTHS"] = df_am_capetown["TENURE"].str.extract(r"([-+]?\d*\.?\d+)").astype(float) * 12
df_am_capetown["TRAININGS_COMPLETED"] = df_am_capetown["TRAININGS_COMPLETED"].str.extractall(r"Level\s*(\d+)").astype(int).groupby(level=0)[0].apply(list)
df_am_capetown["CONTACT_CENTRE"] = "Capetown"
df_am_capetown_formatted = df_am_capetown[["AGENT_ID", "EMPLOYMENT_TYPE", "TRAININGS_COMPLETED", "TENURE_MONTHS", "CONTACT_CENTRE"]]
df_am_capetown_formatted = unnest_training_completed(df_am_capetown_formatted)

df_am_manila["TENURE_MONTHS"] = df_am_manila["TENURE"].str.extract(r"([-+]?\d*\.?\d+)").astype(float)
df_am_manila["TRAININGS_COMPLETED"] = df_am_manila["TRAININGS_COMPLETED"].str.extractall(r"L(\d+)").astype(int).groupby(level=0)[0].apply(list)
df_am_manila["CONTACT_CENTRE"] = "Manila"
df_am_manila_formatted = df_am_manila[["AGENT_ID", "EMPLOYMENT_TYPE", "TRAININGS_COMPLETED", "TENURE_MONTHS", "CONTACT_CENTRE"]]
df_am_manila_formatted = unnest_training_completed(df_am_manila_formatted)

df_am_toronto["TRAININGS_COMPLETED"] = df_am_toronto["TRAININGS_COMPLETED"].str.extractall(r"L(\d+)").astype(int).groupby(level=0)[0].apply(list)
df_am_toronto["CONTACT_CENTRE"] = "Toronto"
df_am_toronto_formatted = df_am_toronto[["AGENT_ID", "EMPLOYMENT_TYPE", "TRAININGS_COMPLETED", "TENURE_MONTHS", "CONTACT_CENTRE"]]
df_am_toronto_formatted = unnest_training_completed(df_am_toronto_formatted)

df_combined = pd.concat([df_am_budapest_formatted, df_am_capetown_formatted, df_am_manila_formatted, df_am_toronto_formatted]).reset_index(drop=True)

In [ ]:
load_into_snowflake(df_combined, "EA_INSIGHTS_ASSESSMENT.EA_INSIGHTS_SCHEMA.AGENT_MASTER_COMBINED")